In [104]:
import warnings
import pandas as pd
import numpy as np

from jre_utils.datapath import factor_data_paths, DATA_DIRECTORY_PATH

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [138]:
migration_unprocessed_path = factor_data_paths["unprocessed"]["migration"]["municipality"]
additional_migration_unprocessed_path = f"{DATA_DIRECTORY_PATH}/municipality/unprocessed/migration/migration_2020_2022.csv"
new_migration_unprocessed_path = f"{DATA_DIRECTORY_PATH}/municipality/unprocessed/migration/migration_1996_2022.csv"

migration_df = pd.read_csv(migration_unprocessed_path)
additional_migration_df = pd.read_csv(additional_migration_unprocessed_path)

In [133]:
prefectures = migration_df["area"].str.split(" ", expand=True)[0].unique() # 47 prefectures

additional_migration_df["Prefecture"] = np.where(
    additional_migration_df["area"].isin(prefectures),
    additional_migration_df["area"],
    np.NaN,
)
additional_migration_df["Prefecture"] = additional_migration_df["Prefecture"].ffill()
additional_migration_df["area"] = (
    additional_migration_df["Prefecture"] + " " + additional_migration_df["area"]
)

In [134]:
existing_areas = migration_df["area"].unique()

discarded_migration_df = additional_migration_df[~additional_migration_df["area"].isin(existing_areas)]
additional_migration_df = additional_migration_df[additional_migration_df["area"].isin(existing_areas)]
additional_migration_df = additional_migration_df[additional_migration_df["year"] == 2022].drop(columns=["Prefecture", "net_migrations"])

In [128]:
additional_migration_df

,year,area,in_migrations,out_migrations,net_migrations
6,2022,Hokkaido Sapporo-shi,116950,108037,8913
17,2022,Hokkaido Hakodate-shi,7889,8581,-692
18,2022,Hokkaido Otaru-shi,3032,3320,-288
19,2022,Hokkaido Asahikawa-shi,9623,10004,-381
20,2022,Hokkaido Muroran-shi,2763,3483,-720
...,...,...,...,...,...
2379,2022,Okinawa-ken Kumejima-cho,402,511,-109
2380,2022,Okinawa-ken Yaese-cho,1706,1217,489
2382,2022,Okinawa-ken Tarama-son,59,60,-1
2384,2022,Okinawa-ken Taketomi-cho,580,594,-14


In [136]:
updated_migration_df = pd.concat([additional_migration_df, migration_df])
updated_migration_df

,year,area,in_migrations,out_migrations
6,2022,Hokkaido Sapporo-shi,116950,108037
17,2022,Hokkaido Hakodate-shi,7889,8581
18,2022,Hokkaido Otaru-shi,3032,3320
19,2022,Hokkaido Asahikawa-shi,9623,10004
20,2022,Hokkaido Muroran-shi,2763,3483
...,...,...,...,...
49810,1996,Okinawa-ken Kumejima-cho,549,619
49811,1996,Okinawa-ken Yaese-cho,1539,1212
49812,1996,Okinawa-ken Tarama-son,52,98
49813,1996,Okinawa-ken Taketomi-cho,430,441


In [139]:
updated_migration_df.to_csv(new_migration_unprocessed_path, index=False)
